# Dataset preparation notebook

## Load data



In [2]:
import pandas as pd
import numpy as np

In [3]:
# Just pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/var/folders/hx/0dxb0wn51wx5pscstjkwf2gc000qqf/T/ipykernel_33025/1119663756.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [24]:
tweets_df = pd.read_csv('../data/analysed_tweets/ukraine_russia/tweets_raw_en_df_individual_5000dailytweets_2022-02-10_to_2022-03-10_sentimentLDA.csv')

In [25]:
tweets_df.head(100)
tweets_df.shape

(112274, 25)

## Filter out any organisations that are in our list 

This we have done to the dataset before but the list may have been expanded so might as well do it again>

In [26]:
organisations_file_path = "../data/organisations.csv"
organisations_list = list(np.genfromtxt(organisations_file_path, delimiter=',', dtype=str, encoding='utf-8'))

# Drop any NaN rows first
tweets_df = tweets_df.dropna(subset=['Display Name'], how='all')        
tweets_df.reset_index(drop=True, inplace=True)
    
for organisation_name in organisations_list:
    tweets_df = tweets_df.drop(tweets_df[tweets_df['Display Name'].str.lower().str.contains(str.lower(organisation_name))].index)


tweets_df = tweets_df.reset_index(drop=True)

In [27]:
tweets_df.shape

(111244, 25)

## Filter out any accounts which do not include human names

This is a stricter filter that will filter out any accounts whose name is not within a list of human names in the latin alphabet. The ames file includes names from many cultures, but it may have been better just to use English ones to avoid cases like 'You' being included and so not filtering out things like 'youtube'.

In [28]:
names_path = '../data/human_names_dict.txt'
with open(names_path, encoding="utf8", errors='ignore') as f:
    lines = f.readlines()

# Names file - I found this one, which includes names from many cultures
# But it may have been better just to use English ones to avoid cases like 'You' being included
# and so not filtering out things like 'youtube'
names_list = []
import string
for idx, line in enumerate(lines[362:]): # This file is so weirdly laid out, really!
    name = line[3:].split(' ')[0]
    char_set = string.ascii_letters
    if all((True if x in char_set else False for x in name)) is True:
        names_list.append(name)

# Filters out all names not in the name list
# Fun aside - if you try to do this in a loop, it takes FOREVER
name_mask = tweets_df['Display Name'].str.lower().str.split(' ').str[0].isin([x.lower() for x in names_list])
tweets_df = tweets_df[name_mask]

tweets_df = tweets_df.reset_index(drop=True)

In [29]:
tweets_df.shape

(53474, 25)

In [30]:
tweets_df.to_csv('../data/analysed_tweets/ukraine_russia/tweets_raw_en_df_individual_latin_5000dailytweets_2022-02-10_to_2022-03-10_sentimentLDA_save.csv')